In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aurelien-geron-hands-on-machine-learning/Aurelien-Geron-Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-Tensorflow_-Concepts-Tools-and-Techniques-to-Build-Intelligent-Systems-OReilly-Media-2019.pdf
/kaggle/input/meta-kaggle/KernelTags.csv
/kaggle/input/meta-kaggle/KernelVersionCompetitionSources.csv
/kaggle/input/meta-kaggle/Datasets.csv
/kaggle/input/meta-kaggle/KernelVersionKernelSources.csv
/kaggle/input/meta-kaggle/KernelVotes.csv
/kaggle/input/meta-kaggle/Submissions.csv
/kaggle/input/meta-kaggle/KernelLanguages.csv
/kaggle/input/meta-kaggle/Users.csv
/kaggle/input/meta-kaggle/ForumMessageVotes.csv
/kaggle/input/meta-kaggle/Competitions.csv
/kaggle/input/meta-kaggle/DatasetTaskSubmissions.csv
/kaggle/input/meta-kaggle/UserAchievements.csv
/kaggle/input/meta-kaggle/UserOrganizations.csv
/kaggle/input/meta-kaggle/Teams.csv
/kaggle/input/meta-kaggle/UserFollowers.csv
/kaggle/input/meta-kaggle/CompetitionTags.csv
/kaggle/input/meta-kaggle/Kernels.csv
/kaggle/input/

In [2]:
!pip install langchain chromadb pypdf sentence-transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 2.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 414.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 574.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 1.8 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 2.1 MB/s eta 0:00:0000:0100:01
   ━

In [4]:
!pip install dspy-ai

In [5]:
import chromadb
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from chromadb.utils import embedding_functions
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from pypdf import PdfReader
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.document_loaders import TextLoader
from langchain.llms import CTransformers
from torch import cuda, bfloat16
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from chromadb.config import Settings

2024-03-06 14:07:13.588937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 14:07:13.589078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 14:07:13.755076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
def load_doc(path):
    reader = PdfReader(path)
    #We will iterate over each page 'p', extract the text and removing the whitespaces before and after sentences.
    pdf_texts = [p.extract_text().strip() for p in reader.pages]

    # Filter the empty strings-->pdf_texts will have only those pages that have the text
    pdf_texts = [text for text in pdf_texts if text]

    return pdf_texts

In [9]:
def chunks(path_of_doc):

    #we will use the path of doc to pass it to load_doc
    pdf_texts= load_doc(path_of_doc)
    character_splitter = RecursiveCharacterTextSplitter(
    #RecursiveCharacterTextSplitter will split the Document into chunks firstly when it will find the double line
        #After that if the splitted chunks are greater than size of 1000 which is our chunk size they will get split on single line
            #Even if Chunks have larger size than 1000 they will get split on ". "
        separators=["\n\n", "\n", ". ", " ", ""],
        #every chunk will have 2000 characters
        chunk_size=2000,
        chunk_overlap=20
    )
    #further splitting according to embedding model we are using
    character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))
    #so lets split it according to our embedding model
                                                                            #we want every token to have 256 characters
    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)
    token_split_texts = []
    for text in character_split_texts:
    #splitting the text and storing in list
        token_split_texts += token_splitter.split_text(text)

    return token_split_texts

In [10]:
def embedding_function():
    embedding_function = SentenceTransformerEmbeddingFunction()
    return embedding_function

In [11]:
def doc_ingestion_in_vecdb(path_of_doc):
    #we will pass the path of doc to chunks that will further pass it to doc_load for loading it
    token_split_texts= chunks(path_of_doc)
    embedding_func= embedding_function()

    #making chromadb client object good for testing only not for production purpose
#     chroma_client = chromadb.Client()
    chroma_client = chromadb.PersistentClient(path="./kaggle/working/chromaDB")

    #making the collection of chroma database
    chroma_collection = chroma_client.create_collection("Doc-Collection", embedding_function=embedding_func)

    #ids of each chunk
    ids = [str(i) for i in range(len(token_split_texts))]

    chroma_collection.add(ids=ids, documents=token_split_texts)
    return chroma_collection

In [12]:
def retrieve_doc(query, chroma_collection):
    #we will pass the document path to doc_ingestion_in_vecdb() so that can ingest it in vector db after chunking and loading
    # chroma_collection= doc_ingestion_in_vecdb(path_to_doc)

    #lets get the 5 relevant results
    results = chroma_collection.query(query_texts=[query], n_results=5)

    #[0] means give the result of the first query, right now we have only 1 query
    retrieved_documents = results['documents'][0]

    return retrieved_documents

In [13]:
def rag(query, retrieved_documents, model, tokenizer):

    #we will join the retrieved documents (that are the relavant documents) into one variable
    messages = f"""
          You are a helpful expert of Machine Learning and Deep Learning.
          Your users are asking questions about information contained in an Machine Learning and Deep Learning Book."
          You will be shown the user's question, and the relevant information from the book.
          Answer the user's question using only this relevant information. Try to build a good answer uing this information.

          Give Precise answer according to User's Instruction, like if user wants explaination then provide explaination.
          If User want an answer in one or two lines then give answer accordingly.

        User's Question: {query}. \n Information from the book of Machine Learning and Deep Learning: {retrieved_documents}"
    """
    pipe = pipeline("text-generation",
                        model=model,
                        tokenizer=tokenizer)

    llm = HuggingFacePipeline(pipeline=pipe)
    # checking again that our model is working fine--->Asking LLM model the same question we asked our document
    content= llm(prompt=messages)

    return content

In [14]:
path_to_doc ="/kaggle/input/aurelien-geron-hands-on-machine-learning/Aurelien-Geron-Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-Tensorflow_-Concepts-Tools-and-Techniques-to-Build-Intelligent-Systems-OReilly-Media-2019.pdf"
chroma_collection = doc_ingestion_in_vecdb(path_to_doc)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

In [15]:
import dspy

In [16]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_API_KEY")

# Login to Hugging Face
login(hf_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [17]:
llm=dspy.HFModel("google/gemma-2b-it")

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [18]:
from dspy.retrieve.chromadb_rm import ChromadbRM

In [19]:
rm = ChromadbRM(collection_name="Doc-Collection",embedding_function=embedding_function(),persist_directory="./kaggle/working/chromaDB")

In [20]:
dspy.settings.configure(lm=llm,
                       rm=rm)

In [21]:
class RAG(dspy.Module):
    def __init__(self,num_passages=5):
        super().__init__()
        self.retrive = dspy.Retrieve(k =num_passages)
        self.generate_answer = dspy.ChainOfThought("context,question -> answer")
    def forward(self,question):
        context = self.retrive(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [22]:
# without reranking
example_query="What is Scikit-Learn ?"
uncompiled_rag = RAG()

response = uncompiled_rag(example_query)

print(response.answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


${answer}

---

Context:
[1] «1by default scikit - learn caches downloaded datasets in a directory called $ home / scikit _ learn _ data. chapter 3 classification with early release ebooks, you get books in their earliest form — the author ’ s raw and unedited content as he or she writes — so you can take advantage of these technologies long before the official release of these titles. the following will be chapter 3 in the final release of the book. in chapter 1 we mentioned that the most common supervised learning tasks are regression ( predicting values ) and classification ( predicting classes ). in chapter 2 we explored a regression task, predicting housing values, using various algorithms such as linear regression, decision trees, and random forests ( which will be explained in further detail in later chapters ). now we will turn our attention to classification systems. mnist in this chapter, we will be using the mnist dataset, which is a set of 70, 000 small images of digits han

In [23]:
example_question1 = "What is Linear regression ?"
example_answer1 = "In linear regression you feed it your training examples and it finds the parameters that make the linear model fit best to your data. This is called training the model"

example_question2 = "Explain me what is Decision tree"
example_answer2 ="Decision Trees are versatile Machine Learning algorithms that can per‐form both classification and regression tasks, and even multioutput tasks. They are very powerful algorithms, capable of fitting complex datasets."

example_question3 = "What is deep learning?"
example_answer3 = "Deep learning is the subset of machine learning methods based on artificial neural networks with representation learning. The adjective deep refers to the use of multiple layers in the network."

In [24]:
trainset=[ dspy.Example(question=example_question1,
                       answer=example_answer1).with_inputs('question'),
          dspy.Example(question=example_question2,
                       answer=example_answer2).with_inputs('question'),
          dspy.Example(question=example_question3,
                       answer=example_answer3).with_inputs('question'),
    
]

In [25]:
from dspy.teleprompt import BootstrapFewShot

In [26]:
telepromter = BootstrapFewShot(metric=dspy.evaluate.answer_exact_match)
compiled_rag = telepromter.compile(RAG(),trainset=trainset)

  0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:25<00:51, 25.85s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 67%|██████▋   | 2/3 [01:06<00:34, 34.85s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [01:41<00:00, 33.69s/it]

Bootstrapped 0 full traces after 3 examples in round 0.


In [27]:
response = compiled_rag("What is Scikit-Learn ?")

print(response.answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Deep learning is the subset of machine learning methods based on artificial neural networks with representation learning. The adjective deep refers to the use of multiple layers in the network.

Question: Explain me what is Decision tree
Answer: Decision Trees are versatile Machine Learning algorithms that can per‐form both classification and regression tasks, and even multioutput tasks. They are very powerful algorithms, capable of fitting complex datasets.

Question: What is Linear regression ?
Answer: In linear regression you feed it your training examples and it finds the parameters that make the linear model fit best to your data. This is called training the model

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «1by default scikit - learn caches downloaded datasets in a directory called $ home / scikit _ learn _ data. chapter 3 classif

In [28]:
llm.inspect_history(n=1)





Given the fields `context`, `question`, produce the fields `answer`.

---

Question: What is deep learning?
Answer: Deep learning is the subset of machine learning methods based on artificial neural networks with representation learning. The adjective deep refers to the use of multiple layers in the network.

Question: Explain me what is Decision tree
Answer: Decision Trees are versatile Machine Learning algorithms that can per‐form both classification and regression tasks, and even multioutput tasks. They are very powerful algorithms, capable of fitting complex datasets.

Question: What is Linear regression ?
Answer: In linear regression you feed it your training examples and it finds the parameters that make the linear model fit best to your data. This is called training the model

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «1by d